In [3]:
from langchain_community.document_loaders import PyPDFLoader
lodaer = PyPDFLoader("attention.pdf")
documents = lodaer.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 20)
text_splitter.split_documents(documents)[:5]

[Document(metadata={'source': 'attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\n

In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embedding  = OllamaEmbeddings(
    model= "llama2"
)


db = FAISS.from_documents(documents[:30] , embedding)


In [6]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000184282388E0>)

In [7]:
# Selecting LLm
from langchain_community.llms import Ollama
llm = Ollama(model =  "llama2")

In [11]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(""" 
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm , prompt)

In [13]:
from langchain.chains import create_retrieval_chain
retrievel_chain = create_retrieval_chain(retriever , document_chain)

In [16]:
response = retrievel_chain.invoke({"input" : "What was new in Attention all you need paper , explain me simple words"})

In [18]:
response["answer"]

'The "Attention is All You Need" paper introduced several new ideas and innovations in the field of natural language processing (NLP):\n\n1. Multi-head Attention: The paper proposed a new attention mechanism called multi-head attention, which allows the model to jointly attend to information from different representation subspaces at different positions. This allows the model to capture longer-range dependencies and better handle input sequences with varying lengths.\n2. Parallel Attention Layers: The authors proposed using multiple parallel attention layers, each with its own set of learnable parameters. This reduces the computational cost of attention while maintaining its expressive power.\n3. Position-wise Feed-Forward Networks: The paper introduced the use of position-wise feed-forward networks (FFNs) in addition to attention sub-layers. FFNs are used to process each position separately and identically, allowing the model to capture non-linear interactions between tokens.\n4. Lear